# FX Trading Strategy

# Currencies, USD, EUR, GBP, HKG, JPY 

 Convert \ to	USD	EUR	GBP	HKD	JPY
 USD	1	1.01864	0.64090	7.79850	118.55000
 EUR	0.97240	1	0.62950	7.65520	116.41000
 GBP	1.55930	1.58810	1	12.15400	184.97000
 HKD	0.12812	0.13040	0.08210	1	15.10050
 JPY	0.00843	0.00856	0.00540	0.06580	1

 Baldwin’s current portfolio of cash holdings includes 2 million USD, 5 million EUR, 1 million GBP, 3 million HKD,
 and 30 million JPY. This portfolio is equivalent to $9,058,560 USD under the current exchange rates 
 (given earlier). Wes has asked you to design a currency trading plan that would increase Baldwin’s euro and 
 yen holdings to 8 million EUR and 54 JPY, respectively, while maintaining the equivalent of at least $250,000 USD
 in each currency. Baldwin measures transaction costs as the change in the USD equivalent value of the portfolio.

 1. Create a model for this problem and solve it.

 2. What is the optimal trading plan?

 3. What is the optimal transaction cost (in equivalent USD)?

 4. Suppose another executive thinks that holding $250,000 USD in each currency is excessive and wants to lower 
 the amount to $50,000 USD in each currency. Does this help to lower the transaction cost? Why or why not?

 5. Suppose the exchange rate for converting USD to GBP increased from 0.6409 to 0.6414. What happens to the 
 optimal solution in this case?
 
 
 # Assumptions
 - the FX rates stay the same after our Trading (given traded value of Fx market, it will most likely)

 - We can trade currencies like HKG to JPY directly, without going through USD
 - We are using one step converion, not many hops to get Jpy or Eur


# What are our Decision Variables
 We need to increase out JPY and EUR holdings, which means we need to sell some of USD, GBP and HKG to buy EUR and JPY

 X1 = millions of USD sold to buy Jpy, X2 = millions of GBP sold to buy Jpy, X3 = millions of HKG sold for jpy
 X4 = millions of USD sold to buy EUR, X5 = millions of GBP sold to buy EUR, X6 = millions of HKG sold for EUR


30 + X1*usd/jpy + X2*gbp/jpy + X3*Hkg/Jpy >= 54. ## JPY holding
5 + X4*usd/eur + X5*Gbp/Eur + X6*Hkg/Eur >= 8  ## new EUR holding 
(2 -  X1 - X4) >= 0.25 ## in Millions USD
(1 - X2 - X5)*Gbp/Usd  >= 0.25 mill USD
(3 - X3 - X6)*Hkg?Usd  >= 0.25 mill USD 

 # Objective Function, reducing Transaction Cost in USD, LPMinimize
 The Portfolio shrunk by this value , (Original value - new Value)
 
2000000 + 5000000*0.97240 + 1000000*1.55930 + 3000000*0.12812 + 30000000*0.00843 - 
(2000000-(X1+X4)*1mill) + 8000000*0.97240 + (1000000-(X2+X5)*1mill)*1.55930 + (3000000-(X3+X6)*1mill)*0.12812 + 54000000*0.00843
 == , cancelling 2mill usd, 1mill Gbp, 3mill Hkd from the equation 

 LPMinimize this value (USD)
(X1+X4)*1mill - 3000000*0.97240 + (X2+X5)*1mill*1.55930 + (X3+X6)*1mill*0.12812 - 24000000**0.00843



In [3]:
# Just making sure I got the convesions correct, to Evaluate the Net value of the portfolio, and thus the
# net Transaction cost 
#2 million USD, 5 million EUR, 1 million GBP, 3 million HKD, and 30 million JPY == $9,058,560 USD
2000000 + 5000000*0.97240 + 1000000*1.55930 + 3000000*0.12812 + 30000000*0.00843

9058560.0

In [6]:
# import pulp
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize


# Problem FX Trading 
# define variables
USD_SOLD_JPY = LpVariable("USD_SOLD_FOR_JPY", 0, None) # Tables>=0   X1 
GBP_SOLD_JPY = LpVariable("GBP_SOLD_FOR_JPY", 0, None) # Tables>=0   X2 
HKG_SOLD_JPY = LpVariable("HKG_SOLD_FOR_JPY", 0, None) # Tables>=0   X3

USD_SOLD_EUR = LpVariable("USD_SOLD_FOR_EUR", 0, None) # Tables>=0   X4
GBP_SOLD_EUR = LpVariable("GBP_SOLD_FOR_EUR", 0, None) # Tables>=0   X5
HKG_SOLD_EUR = LpVariable("HKG_SOLD_FOR_EUR", 0, None) # Tables>=0   X6



# defines the problem
prob = LpProblem("problem", LpMinimize)   # Minimize the Transaction Cost  
# Note, LpMaximize for a maximization problem, 
# and LpMinimize for a minimization problem

# Constraints
#30 + X1*usd/jpy + X2*gbp/jpy + X3*Hkg/Jpy >= 54. ## JPY holding
#5 + X4*usd/eur + X5*Gbp/Eur + X6*Hkg/Eur >= 8  ## new EUR holding 
#(2 -  X1 - X4) >= 0.25 ## in Millions USD
#(1 - X2 - X5)*Gbp/Usd  >= 0.25 mill USD
#(3 - X3 - X6)*Hkg?Usd  >= 0.25 mill USD

# define constraints
prob += 30000000 + USD_SOLD_JPY*118.55000 + GBP_SOLD_JPY*184.97000  + HKG_SOLD_JPY*15.10050  >= 54000000   # Target JPY Holding 
prob += 5000000 + USD_SOLD_EUR*1.01864   + GBP_SOLD_EUR*1.58810   +  HKG_SOLD_EUR*0.13040    >= 8000000

prob += 2000000 - USD_SOLD_JPY - USD_SOLD_EUR >= 250000
prob += (1000000 - GBP_SOLD_JPY - GBP_SOLD_EUR)*1.55930 >= 250000
prob += (3000000 - HKG_SOLD_JPY - HKG_SOLD_EUR)*0.12812 >= 250000




# define objective function, which is the Transaction Cost Minimize
# This Portfolio value before - porfolio value after 
prob += USD_SOLD_JPY + USD_SOLD_EUR  - 3000000*0.97240 + (GBP_SOLD_JPY+GBP_SOLD_EUR)*1.55930 + (HKG_SOLD_JPY+HKG_SOLD_EUR)*0.12812 - 24000000**0.00843


# solve the problem
status = prob.solve()
print(f"Problem FX Tradiing")
print(f"status={LpStatus[status]}")

# print the results
for variable in prob.variables():
    #print(f"{variable.name} = {round(variable.varValue)}")
    print(f"{variable.name} = {variable.varValue}")
  
print(f"Objective Tx Cost = {value(prob.objective)}")
print(f"")



Problem FX Tradiing
status=Optimal
GBP_SOLD_FOR_EUR = 709920.88
GBP_SOLD_FOR_JPY = 129750.77
HKG_SOLD_FOR_EUR = 689836.31
HKG_SOLD_FOR_JPY = 0.0
USD_SOLD_FOR_EUR = 1750000.0
USD_SOLD_FOR_JPY = 0.0
Objective Tx Cost = 230480.6778572969



In [29]:
# Decreasing the Minimum portfolio holdingd to 50000 USD
# This decreases the Transaction Cost a little 
# Decreasing the resver amount to 50K, allows us to spend mor USD to get EUR, as we can get more EUR with USD,
# this helps bring down the transaction cost


# define variables
USD_SOLD_JPY = LpVariable("USD_SOLD_FOR_JPY", 0, None) # Tables>=0   X1 
GBP_SOLD_JPY = LpVariable("GBP_SOLD_FOR_JPY", 0, None) # Tables>=0   X2 
HKG_SOLD_JPY = LpVariable("HKG_SOLD_FOR_JPY", 0, None) # Tables>=0   X3

USD_SOLD_EUR = LpVariable("USD_SOLD_FOR_EUR", 0, None) # Tables>=0   X4
GBP_SOLD_EUR = LpVariable("GBP_SOLD_FOR_EUR", 0, None) # Tables>=0   X5
HKG_SOLD_EUR = LpVariable("HKG_SOLD_FOR_EUR", 0, None) # Tables>=0   X6

# defines the problem
prob = LpProblem("problem", LpMinimize)   # Minimize the Transaction Cost  

# define constraints
prob += 30000000 + USD_SOLD_JPY*118.55000 + GBP_SOLD_JPY*184.97000  + HKG_SOLD_JPY*15.10050  >= 54000000   # Target JPY Holding 
prob += 5000000 + USD_SOLD_EUR*1.01864   + GBP_SOLD_EUR*1.58810   +  HKG_SOLD_EUR*0.13040    >= 8000000

prob += 2000000 - USD_SOLD_JPY - USD_SOLD_EUR >= 50000
prob += (1000000 - GBP_SOLD_JPY - GBP_SOLD_EUR)*1.55930 >= 50000
prob += (3000000 - HKG_SOLD_JPY - HKG_SOLD_EUR)*0.12812 >= 50000


# define objective function, which is the Transaction Cost Minimize
# This Portfolio value before - porfolio value after 
prob += USD_SOLD_JPY + USD_SOLD_EUR  - 3000000*0.97240 + (GBP_SOLD_JPY+GBP_SOLD_EUR)*1.55930 + (HKG_SOLD_JPY+HKG_SOLD_EUR)*0.12812 - 24000000**0.00843

# solve the problem
status = prob.solve()
print(f"Problem FX Tradiing")
print(f"status={LpStatus[status]}")

# print the results
for variable in prob.variables():
    #print(f"{variable.name} = {round(variable.varValue)}")
    print(f"{variable.name} = {variable.varValue}")
  
print(f"Objective Tx Cost = {value(prob.objective)}")
print(f"")

Problem FX Tradiing
status=Optimal
GBP_SOLD_FOR_EUR = 638279.71
GBP_SOLD_FOR_JPY = 129750.77
HKG_SOLD_FOR_EUR = 0.0
HKG_SOLD_FOR_JPY = 0.0
USD_SOLD_FOR_EUR = 1950000.0
USD_SOLD_FOR_JPY = 0.0
Objective Tx Cost = 230388.77343909675



In [30]:
# USD/GBP increased from 0.6409 to 0.6414  , Dollar became stronger 
# The Optimal soltion changes a little, and this decreases the Transaction Cost a little too, 
# just by around $176.31

# define variables
USD_SOLD_JPY = LpVariable("USD_SOLD_FOR_JPY", 0, None) # Tables>=0   X1 
GBP_SOLD_JPY = LpVariable("GBP_SOLD_FOR_JPY", 0, None) # Tables>=0   X2 
HKG_SOLD_JPY = LpVariable("HKG_SOLD_FOR_JPY", 0, None) # Tables>=0   X3

USD_SOLD_EUR = LpVariable("USD_SOLD_FOR_EUR", 0, None) # Tables>=0   X4
GBP_SOLD_EUR = LpVariable("GBP_SOLD_FOR_EUR", 0, None) # Tables>=0   X5
HKG_SOLD_EUR = LpVariable("HKG_SOLD_FOR_EUR", 0, None) # Tables>=0   X6

# defines the problem
prob = LpProblem("problem", LpMinimize)   # Minimize the Transaction Cost  

# define constraints
prob += 30000000 + USD_SOLD_JPY*118.55000 + GBP_SOLD_JPY*184.97000  + HKG_SOLD_JPY*15.10050  >= 54000000   # Target JPY Holding 
prob += 5000000 + USD_SOLD_EUR*1.01864   + GBP_SOLD_EUR*1.58810   +  HKG_SOLD_EUR*0.13040    >= 8000000

prob += 2000000 - USD_SOLD_JPY - USD_SOLD_EUR >= 250000
prob += (1000000 - GBP_SOLD_JPY - GBP_SOLD_EUR)*round(1/0.6414, 5) >= 250000
prob += (3000000 - HKG_SOLD_JPY - HKG_SOLD_EUR)*0.12812 >= 250000


# define objective function, which is the Transaction Cost Minimize
# This Portfolio value before - porfolio value after 
prob += USD_SOLD_JPY + USD_SOLD_EUR  - 3000000*0.97240 + (GBP_SOLD_JPY+GBP_SOLD_EUR)*round(1/0.6414, 5) + (HKG_SOLD_JPY+HKG_SOLD_EUR)*0.12812 - 24000000**0.00843

# solve the problem
status = prob.solve()
print(f"Problem FX Tradiing")
print(f"status={LpStatus[status]}")

# print the results
for variable in prob.variables():
    #print(f"{variable.name} = {round(variable.varValue)}")
    print(f"{variable.name} = {variable.varValue}")
  
print(f"Objective Tx Cost = {value(prob.objective)}")
print(f"")

Problem FX Tradiing
status=Optimal
GBP_SOLD_FOR_EUR = 709899.28
GBP_SOLD_FOR_JPY = 129750.77
HKG_SOLD_FOR_EUR = 690099.31
HKG_SOLD_FOR_JPY = 0.0
USD_SOLD_FOR_EUR = 1750000.0
USD_SOLD_FOR_JPY = 0.0
Objective Tx Cost = 230304.3660267968



In [14]:
round(1/0.6409, 5)

1.56031

In [15]:
round(1/0.6414, 5)

1.55909

In [22]:
230480.6778572969 - 230304.3660267968

176.31183050008258

In [1]:
160000 * 1.55930

249487.99999999997

In [3]:
(1000000 - 709920.88 - 129750.77) * 1.55930

249999.99615499994